# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [90]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [91]:
cities = pd.read_csv('Output Data Weather/Cities.csv')
cities = cities.drop("Unnamed: 0", 1)
cities

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,nishihara,75.0,JP,1.569345e+09,88.0,35.74,139.53,75.99,6.93
1,busselton,13.0,AU,1.569345e+09,71.0,-33.64,115.35,55.00,16.31
2,dunedin,49.0,NZ,1.569345e+09,74.0,-45.87,170.50,44.01,1.99
3,cape town,20.0,ZA,1.569345e+09,59.0,-33.93,18.42,70.00,18.34
4,kapaa,90.0,US,1.569345e+09,78.0,22.08,-159.32,80.60,11.41
...,...,...,...,...,...,...,...,...,...
450,saint-denis,77.0,FR,1.569345e+09,87.0,48.94,2.36,64.40,16.11
451,satapuala,81.0,WS,1.569345e+09,94.0,-13.84,-171.98,75.00,5.01
452,guanica,66.0,PR,1.569345e+09,76.0,17.97,-66.91,89.01,3.00
453,benguela,75.0,AO,1.569345e+09,100.0,-12.58,13.40,71.60,4.70


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [92]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [93]:
# Store 'Lat' and 'Lng' into  locations 
locations = cities[["Lat", "Lng"]].astype(float)
locations
center_lat = (cities["Lat"].max() + cities["Lat"].min()) /2
center_lng = (cities["Lng"].max() + cities["Lng"].min()) /2

In [94]:
# Convert Humidity to float and store
cities = cities.dropna()
humidity = cities["Humidity"].astype(float)
max_intensity = humidity.max()

In [95]:
# Create a poverty Heatmap layer
fig = gmaps.figure()
fig = gmaps.figure(center = [center_lat,center_lng], zoom_level=1.8)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity= max_intensity,
                                 point_radius = 5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [96]:
cities = cities.dropna()
cities

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,nishihara,75.0,JP,1.569345e+09,88.0,35.74,139.53,75.99,6.93
1,busselton,13.0,AU,1.569345e+09,71.0,-33.64,115.35,55.00,16.31
2,dunedin,49.0,NZ,1.569345e+09,74.0,-45.87,170.50,44.01,1.99
3,cape town,20.0,ZA,1.569345e+09,59.0,-33.93,18.42,70.00,18.34
4,kapaa,90.0,US,1.569345e+09,78.0,22.08,-159.32,80.60,11.41
...,...,...,...,...,...,...,...,...,...
450,saint-denis,77.0,FR,1.569345e+09,87.0,48.94,2.36,64.40,16.11
451,satapuala,81.0,WS,1.569345e+09,94.0,-13.84,-171.98,75.00,5.01
452,guanica,66.0,PR,1.569345e+09,76.0,17.97,-66.91,89.01,3.00
453,benguela,75.0,AO,1.569345e+09,100.0,-12.58,13.40,71.60,4.70


In [97]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Cloudiness less than 5%
cities_nice = cities[
        (cities['Max Temp'] >= 70) & 
        (cities['Max Temp'] <= 80) &
        (cities['Wind Speed'] <= 10) &
        (cities['Cloudiness'] <= 5) 
      ]

cities_nice = cities_nice.reset_index(drop=True)
cities_nice

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,birao,3.0,CF,1.569345e+09,70.0,10.30,22.78,78.56,0.83
1,ruidoso,1.0,US,1.569345e+09,45.0,33.33,-105.67,78.01,2.68
2,rajula,1.0,IN,1.569345e+09,88.0,21.04,71.44,77.12,6.69
3,nocupetaro,4.0,MX,1.569345e+09,70.0,19.05,-101.15,78.74,3.65
4,huron,1.0,US,1.569345e+09,77.0,44.36,-98.21,75.20,9.10


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [98]:
hotel_df = cities_nice

hotel_df['Hotel Name'] = ""
hotel_df['Address'] = ""
hotel_df['User Ratings'] = ""
hotel_df['Rating'] = ""

hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name,Address,User Ratings,Rating
0,birao,3.0,CF,1.569345e+09,70.0,10.30,22.78,78.56,0.83,,,,
1,ruidoso,1.0,US,1.569345e+09,45.0,33.33,-105.67,78.01,2.68,,,,
2,rajula,1.0,IN,1.569345e+09,88.0,21.04,71.44,77.12,6.69,,,,
3,nocupetaro,4.0,MX,1.569345e+09,70.0,19.05,-101.15,78.74,3.65,,,,
4,huron,1.0,US,1.569345e+09,77.0,44.36,-98.21,75.20,9.10,,,,


In [99]:
# find the closest restaurant of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = { 
    "rankby": "prominence",
    "type": "lodging", # hotel is not a supported type
    "key": g_key,
    "radius": 20000, # in meters
    "keyword" : "hotel"
}
# Parameters 
# https://developers.google.com/places/web-service/search

In [100]:
i = 0 

for index, row in hotel_df.iterrows():
#    if i == 0:
        # add keyword to params dict       
        lat = hotel_df.iloc[index]['Lat']
        long = hotel_df.iloc[index]['Lng']
        location = f'{lat}, {long}'
        params['location'] = location

        # assemble url and make API request
        city = hotel_df.iloc[index]['City']
        print(f"Retrieving Results for City '{city}'")
        response = requests.get(base_url, params=params).json()
    
        # Pretty print json to find the results
        #print(json.dumps(response, indent=4, sort_keys=True))

        # extract results
        results = response['results']
    
        try:
            print(f"Closest Hotel to {city} is {results[0]['name']}.")
        
            hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
            hotel_df.loc[index, 'Address'] = results[0]['vicinity']
            hotel_df.loc[index, 'User Ratings'] = results[0]['user_ratings_total']
            hotel_df.loc[index, 'Rating'] = results[0]['rating']
        
        except (KeyError, IndexError):
            print("Missing field/result... skipping.")
        
        print("------------")
hotel_df

Retrieving Results for City 'birao'
Missing field/result... skipping.
------------
Retrieving Results for City 'ruidoso'
Closest Hotel to ruidoso is Hotel Ruidoso.
------------
Retrieving Results for City 'rajula'
Closest Hotel to rajula is Hotel Darshan.
------------
Retrieving Results for City 'nocupetaro'
Closest Hotel to nocupetaro is Hotel Juana.
------------
Retrieving Results for City 'huron'
Closest Hotel to huron is Crossroads Hotel-Huron Event Center.
------------


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name,Address,User Ratings,Rating
0,birao,3.0,CF,1.569345e+09,70.0,10.30,22.78,78.56,0.83,,,,
1,ruidoso,1.0,US,1.569345e+09,45.0,33.33,-105.67,78.01,2.68,Hotel Ruidoso,"110 Chase St, Ruidoso",270,4.3
2,rajula,1.0,IN,1.569345e+09,88.0,21.04,71.44,77.12,6.69,Hotel Darshan,"8E Bhavnagar Veraval Highway, Nr. Pipavav Port...",629,4.2
3,nocupetaro,4.0,MX,1.569345e+09,70.0,19.05,-101.15,78.74,3.65,Hotel Juana,"Centro, Carácuaro de Morelos",3,3.7
4,huron,1.0,US,1.569345e+09,77.0,44.36,-98.21,75.20,9.10,Crossroads Hotel-Huron Event Center,"100 4th St SW, Huron",104,3.8


In [101]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in cities_nice.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [102]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, hover_text="", label="", info_box_content=hotel_info, display_info_box=None)

fig.add_layer(marker_layer)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))